In [1]:
from enum import Enum
from typing import NamedTuple

import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pytesseract
from pyaspeller import YandexSpeller

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams['figure.subplot.left'] = 0.1
plt.rcParams['figure.subplot.right'] = 0.99
plt.rcParams['figure.subplot.top'] = 0.97
plt.rcParams['figure.subplot.bottom'] = 0.05
plt.rcParams['figure.subplot.hspace'] = 0.3
matplotlib.rc("image", cmap="gray_r")

# Prepare image
Загрузка, подготовка изображения для распознавания

In [3]:
# choose tool for show image

# Comment it if not use it
%matplotlib qt


class ShowTool(Enum):
    cv2= 'cv2'
    plt='plt'

SHOW_STYLE = ShowTool.plt

In [4]:
match SHOW_STYLE:
    case ShowTool.plt:

        def show(image: np.ndarray, y_size: int = 960):
            """
            Show image in cv2 window

            :param image: Image to show
            :param y_size: Window scaler, defaults to 960
            """
            plt.imshow(image)

    case ShowTool.cv2:

        def show(image: np.ndarray, y_size: int = 960):
            """
            Show image in cv2 window

            :param image: Image to show
            :param y_size: Window scaler, defaults to 960
            """

            dy, dx = image.shape[:2]
            size = (int(dx * y_size / dy), y_size)

            cv2.imshow('contours', cv2.resize(image, size))
            cv2.resizeWindow('contours', *size)
            cv2.waitKey()

## Load
Загружаем изображение и проверяем, правильное ли мы выбрали

In [56]:
img_real = cv2.imread(r"Data\3.jpg")
y, x, _ = img_real.shape
show(img_real)

## Align
Выравниваем (насколько возможно) изображение

In [11]:
def downscale_image(image: np.ndarray, max_size: int = 1920) -> np.ndarray:
    """
    Downscale image

    :param image: Input image
    :param max_size: Max size, defaults to 2048
    :return: Downsized image
    """

    # if max_size <= max_size:
    #     return image

    scale = max_size / max(image.shape)
    return cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)


def make_square(image: np.ndarray) -> np.ndarray:
    """
    Make square from image to rotate it without a border cropping

    :param image: Supplemented image to square
    :return: Image
    """
    y, x = image.shape[:2]
    max_side = max(y, x)

    dy = max_side - y
    dx = max_side - x

    top = dy // 2
    bottom = dy - top
    left = dx // 2
    right = dx - left
    return cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])


def align(image: np.ndarray, tol: float = 5) -> np.ndarray:
    """
    Align image

    :param image: Image to align
    :param tol: Allowable angle deviation, defaults to 5
    :return: Aligned image
    """
    image_processed = cv2.Canny(image, 100, 200)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    image_processed = cv2.dilate(image_processed, kernel, iterations=2)

    non_zero_coordinates = cv2.findNonZero(image_processed)
    box = cv2.minAreaRect(non_zero_coordinates)
    (x, y), angle = box[1:]

    if (-tol < angle < tol) or (90 - tol < angle < 90 + tol) or (angle < -90 + tol):
        return image

    # FIXME: choose right
    # y, x = image.shape
    rotate_M = cv2.getRotationMatrix2D((x // 2, y // 2), angle, 1)
    return cv2.warpAffine(
        image.copy(),
        rotate_M,
        (int(x), int(y)),
        cv2.INTER_CUBIC,
        cv2.BORDER_REPLICATE,
    )

In [57]:
img_gray = cv2.cvtColor(img_real, cv2.COLOR_BGR2GRAY)
img_gray = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
img_gray = downscale_image(img_gray)

# Add little borders to emphasize black spaces on borders
border_add = 10
img_gray = cv2.copyMakeBorder(
    img_gray,
    border_add,
    border_add,
    border_add,
    border_add,
    cv2.BORDER_CONSTANT,
    value=[0, 0, 0],
)
img_gray = make_square(img_gray)
show(img_gray)

# Clasterization
Находим области с текстом о объединяем эти области в кластеры - абзацы для сохранения порядка слов при распознавания и исключения разрывов текста по середине абзаца.

Находим границы объектов, чтобы не включать области у краев, шум и тд. Размываем изображение гауссовским ядром и применяем расширение для образования связанных блоков текста, формируя маску для каждого блока текста 

In [58]:
def prepare_image(image: np.ndarray) -> np.ndarray:
    """Prepare image to clustering"""

    bordered_image = cv2.Canny(image, 100, 200)
    blur_kernel = (9, 9)
    blur_image = cv2.GaussianBlur(bordered_image, blur_kernel, 2)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    dilate_image = cv2.dilate(blur_image, kernel, iterations=10)
    return dilate_image


def split(image: np.ndarray)-> dict[int, np.ndarray]:
    """
    Split image to connected blocks

    :param image: _description_
    """
    n_rectangles, segmented_mask = cv2.connectedComponents(image)
    cluster_list = [np.uint8(segmented_mask == i) * i for i in range(1, n_rectangles + 1)]
    return dict(
        [
            (np.max(mask), mask)
            for mask in sorted(cluster_list, key=lambda i: np.count_nonzero(i), reverse=True)
        ]
    )

In [59]:
prep_image = prepare_image(img_gray)

In [60]:
a=split(prep_image)

In [61]:
show(sum(list(a.values())))

# Recognize
Распознаем текст на изображении и проверяем правильности написания

In [33]:
speller = YandexSpeller()

for split in clusters:


In [31]:
a= AttentionArea(segmented_mask=img_gray,focus_image=clusters[1])
img_cropped = img_gray[a.focus_box.y0:a.focus_box.y1,a.focus_box.x0:a.focus_box.x1]

In [48]:
# img_cropped = cv2.bitwise_and(img_gray, img_gray, mask=clasters[1])
plt.imshow(align(img_cropped),tol=)
text = pytesseract.image_to_string(
    align2(img_cropped),
    lang='rus',
    config='--psm 3',
)
print(text)
print('*' * 50)
print(speller.spelled(text))

В Строй Орнсте мы оформлялн сертификат 150 9001 для участия в
тендере. Нас сразу привлекла стоямость в 19 тысяч рублей. И срочность:
оформлекия сертификата, так как срохи у нас «горели». По телефону:
получили грамотную консультецк по 15 и скан документа был у нас в
ээтот же день. В результате тендер мы вынграли и успешно работзем.
Спасибо СтройЮрнст, за качественную работу!

**************************************************
В Строй Орнсте мы оформляли сертификат 150 9001 для участия в
тендере. Нас сразу привлекла стоимость в 19 тысяч рублей. И срочность:
оформлекия сертификата, так как срохи у нас «горели». По телефону:
получили грамотную консультацию по 15 и скан документа был у нас в
этот же день. В результате тендер мы выиграли и успешно работаем.
Спасибо Стройюрист, за качественную работу!



In [53]:
class RecognizeResult(NamedTuple):
    text: str
    angle: int
    x: int
    y: int
    dx: int
    dy: int


res = []
for img_mask in clusters:

    img_cropped = cv2.bitwise_and(img_gray, img_gray, mask=img_mask)
    non_zero_coords = cv2.findNonZero(img_mask)
    box_cordinates = cv2.boundingRect(non_zero_coords)

    # plt.imshow(img_cropped)

    for angle in [0, -90, 90, 180]:
        text = pytesseract.image_to_string(img_cropped, lang='rus', config='--psm 3')
        text = speller.spelled(text)
        is_correct = len(text)>0
        text = text + '\n'
        if len(text) and is_correct:
            res.append(RecognizeResult(text, angle, *box_cordinates))
            break

In [54]:
angles = [i.angle for i in res]
general_angle = max(set(angles), key=angles.count)
slope = 2
metric = {
    0: lambda f: f. x + slope * f.y,
    -90: lambda f: slope * f.x - (f.y + f.dy),
    90: lambda f: -slope * (f.x + f.dx) + f.y,
    180: lambda f: -(f.x + f.dx) - slope * (f.y + f.dy),
}[general_angle]

In [55]:
print(' '.join([i.text for i in sorted(res, key=metric)]))

Еврогрупп Общество с ограниченной ответственностью.

«Торговая Компания «ЕвроГруп»
ол сковороду Борская 17 ИННЛЕТИБ259 ООЧ2960
огтниАЗеря , ОКЛОРРУНЕ ЛО «НБД-Банко г.Н- Новгороя,БИКОИ2202705
озн за ЧООО ОО 050022557 Корусченуо 1014 О.00ООДООС
"Б-тай: счгостир$2@упай пуТелефонифанс: $ (831) 253-98-60,253-37-91

 № 364 от « 28» мая 2015 г

 ©ОО«СтройЮрист»

 В Строй!Орнсте мы оформляли сертификат 15О 9001 для участия в
тендере. Нас сразу привлекла стоимость в 19 тысяч рублей. И прочность
‘оформяения сертнфиката, та как срокн у нас «горелн». ЛЮ телефону
получили грамотную консультацию по Т5 и ская документа был у ас в
‘этот же день, В результате тендер мы выиграли н успещно работаем.
Спасибо СтройЮрнся, за качественную работу?


 отзыВ

 С уважением,
Директор.

©0О «ТК Еврогру Белов Е.А





In [20]:
plt.imshow(img_gray)